# Example 2.3 from the book: Power System State Estimation - Theory and Implementation, Abur and Gómez

## Network Data

| From bus  | To bus    | R (pu)    | X (pu)    | Total line Charging Suceptance    |
|-----------|--------	|--------	|--------	|--------                           |
| 1         | 2         | 0.01      | 0.03      | 0.00                              |
| 1         | 3         | 0.02      | 0.05      | 0.00                              |
| 2         | 3         | 0.03      | 0.08      | 0.00                              |

## Measurement values and their associated error standard deviations
| Measurement, i    | Type      | Value (pu)    | $\sqrt{R_{ii}}$ |
|----------         |-------    |--------	    |-------          |
| 1                 | $p_{12}$  | 0.8888        | 0.008           |
| 2                 | $p_{13}$  | 0.8888        | 0.008           |
| 3                 | $p_{2}$   | 0.8888        | 0.010           |
| 4                 | $q_{12}$  | 0.8888        | 0.008           |
| 5                 | $q_{13}$  | 0.8888        | 0.008           |
| 6                 | $q_{2}$   | 0.8888        | 0.010           |
| 7                 | $V_{1}$   | 0.8888        | 0.004           |
| 8                 | $V_{2}$   | 0.8888        | 0.004           |

### Define variables

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Latex

a = 0.98
z_12 = 0.01 + 0.03j
y_12 = 1/z_12
g_12 = np.real(y_12)
b_12 = np.imag(y_12)

z_13 = 0.02 + 0.05j
y_13 = 1/z_13
g_13 = np.real(y_13)
b_13 = np.imag(y_13)

z_23 = 0.03 + 0.08j
y_23 = 1/z_23
g_23 = np.real(y_23)
b_23 = np.imag(y_23)

P_12 = 0.888
P_13 = 1.173
P_2 = -0.501
Q_12 = 0.568
Q_13 = 0.663
Q_2 = -0.286
V_1 = 1.006
V_2 = 0.968

z = np.array([P_12, P_13, P_2, Q_12, Q_13, Q_2, V_1, V_2])

display(Latex(f'$z=$ {z} '))

print(g_12)


## Measurement Jacobian

In [2]:
m = 8
n = 5

theta_1 = 0

theta_2_0 = 0
theta_3_0 = 0
V_1_0 = 1.0
V_2_0 = 1.0
V_3_0 = 1.0

theta_12_0 = theta_1 - theta_2_0
theta_13_0 = theta_1 - theta_3_0
theta_23_0 = theta_2_0 - theta_3_0

H_0 = np.zeros((m,n))

x_0 = np.array([theta_2_0, theta_3_0, V_1_0, V_2_0, V_3_0])

### Elements corresponding to real power flow measurements:

$\frac{\partial P_{ij}}{\theta_i} = -V_i V_j(g_{ij}\sin{\theta_{ij}} - b_{ij}\cos{\theta_{ij}})$

$\frac{\partial P_{ij}}{\theta_j} = -V_i V_j(g_{ij}\sin{\theta_{ij}} - b_{ij}\cos{\theta_{ij}})$

$\frac{\partial P_{ij}}{V_i} = -V_j(g_{ij}\cos{\theta_{ij}} + b_{ij}\sin{\theta_{ij}}) + 2(g_{ij}+g_{si})V_i$

$\frac{\partial P_{ij}}{V_j} = -V_i(g_{ij}\cos{\theta_{ij}} + b_{ij}\sin{\theta_{ij}})$

In [3]:
d_p12_d_theta2 = -V_1_0*V_2_0*(g_12*np.sin(theta_12_0) - b_12*np.cos(theta_12_0))
d_p12_d_theta3 = 0
d_p12_d_V1 = -V_2_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0)) + 2*(g_12)*V_1_0
d_p12_d_V2 = -V_1_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0))
d_p12_d_V3 = 0
H_0[0,:] = [d_p12_d_theta2, d_p12_d_theta3, d_p12_d_V1, d_p12_d_V2, d_p12_d_V3]

d_p13_d_theta2 = 0
d_p13_d_theta3 = -V_1_0*V_3_0*(g_13*np.sin(theta_13_0) - b_13*np.cos(theta_13_0))
d_p13_d_V1 = -V_3_0*(g_13*np.cos(theta_13_0) + b_13*np.sin(theta_13_0)) + 2*(g_13)*V_1_0
d_p13_d_V2 = 0
d_p13_d_V3 = -V_1_0*(g_13*np.cos(theta_13_0) + b_13*np.sin(theta_13_0))
H_0[1,:] = [d_p13_d_theta2, d_p13_d_theta3, d_p13_d_V1, d_p13_d_V2, d_p13_d_V3]

### Elements corresponding to real power injection measurements:

$\frac{\partial P_i}{\theta_i} = -\sum_{j=1}^N V_i V_j(-G_{ij}\sin{\theta_{ij}} + B_{ij}\cos{\theta_{ij}}) - V_i^2B_{ii}$

$\frac{\partial P_i}{\theta_j} = -V_i V_j(G_{ij}\sin{\theta_{ij}} - B_{ij}\cos{\theta_{ij}})$

$\frac{\partial P_i}{V_i} = \sum_{j=1}^N V_j(G_{ij}\cos{\theta_{ij}} + B_{ij}\sin{\theta_{ij}}) - V_iG_{ii}$

$\frac{\partial P_i}{V_j} = -V_i(G_{ij}\cos{\theta_{ij}} + B_{ij}\sin{\theta_{ij}})$

In [4]:
d_p2_d_theta_2 = -V_2_0*V_1_0*(-g_12*np.sin(theta_12_0) + b_12*np.cos(theta_12_0)) - V_2_0*V_3_0*(-g_23*np.sin(theta_23_0) + b_23*np.cos(theta_23_0))
d_p2_d_theta_3 = -V_2_0*V_3_0*(g_23*np.sin(theta_23_0) - b_23*np.cos(theta_23_0))
d_p2_d_V1 = -V_2_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0))
d_p2_d_V2 = V_1_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0)) + V_3_0*(g_23*np.cos(theta_23_0) + b_23*np.sin(theta_23_0))
d_p2_d_V3 = -V_2_0*(g_23*np.cos(theta_23_0) + b_23*np.sin(theta_23_0))
H_0[2,:] = [d_p2_d_theta_2, d_p2_d_theta_3, d_p2_d_V1, d_p2_d_V2, d_p2_d_V3]

### Elements corresponding to reactive power flow measurements:

$\frac{\partial Q_{ij}}{\theta_i} = -V_i V_j(g_{ij}\cos{\theta_{ij}} + b_{ij}\sin{\theta_{ij}})$

$\frac{\partial Q_{ij}}{\theta_j} = V_i V_j(g_{ij}\cos{\theta_{ij}} + b_{ij}\sin{\theta_{ij}})$

$\frac{\partial Q_{ij}}{V_i} = -V_j(g_{ij}\sin{\theta_{ij}} - b_{ij}\cos{\theta_{ij}}) - 2V_i(b_{ij}+b_{si})$

$\frac{\partial Q_{ij}}{V_j} = -V_i(g_{ij}\sin{\theta_{ij}} - b_{ij}\cos{\theta_{ij}})$

In [5]:
d_q12_d_theta2 = V_1_0*V_2_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0))
d_q12_d_theta3 = 0
d_q12_d_V1 = -V_2_0*(g_12*np.sin(theta_12_0) - b_12*np.cos(theta_12_0)) - 2*(b_12)*V_1_0
d_q12_d_V2 = -V_1_0*(g_12*np.sin(theta_12_0) - b_12*np.cos(theta_12_0))
d_q12_d_V3 = 0
H_0[3,:] = [d_q12_d_theta2, d_q12_d_theta3, d_q12_d_V1, d_q12_d_V2, d_q12_d_V3]

d_q13_d_theta2 = 0
d_q13_d_theta3 = V_1_0*V_3_0*(g_13*np.cos(theta_13_0) + b_13*np.sin(theta_13_0))
d_q13_d_V1 = -V_3_0*(g_13*np.sin(theta_13_0) - b_13*np.cos(theta_13_0)) - 2*(b_13)*V_1_0
d_q13_d_V2 = 0
d_q13_d_V3 = -V_1_0*(g_13*np.sin(theta_13_0) - b_13*np.cos(theta_13_0))
H_0[4,:] = [d_q13_d_theta2, d_q13_d_theta3, d_q13_d_V1, d_q13_d_V2, d_q13_d_V3]

### Elements corresponding to reactive power injection measurements:

$\frac{\partial Q_i}{\theta_i} = -\sum_{j=1}^N V_i V_j(G_{ij}\cos{\theta_{ij}} + B_{ij}\sin{\theta_{ij}}) - V_i^2G_{ii}$

$\frac{\partial Q_i}{\theta_j} = -V_i V_j(-G_{ij}\cos{\theta_{ij}} - B_{ij}\sin{\theta_{ij}})$

$\frac{\partial Q_i}{V_i} = \sum_{j=1}^N V_j(G_{ij}\sin{\theta_{ij}} - B_{ij}\cos{\theta_{ij}}) - V_iB_{ii}$

$\frac{\partial Q_i}{V_j} = -V_i(G_{ij}\sin{\theta_{ij}} - B_{ij}\cos{\theta_{ij}})$

In [ ]:
d_q2_d_theta_2 = -V_2_0*V_1_0*(g_12*np.cos(theta_12_0) + b_12*np.sin(theta_12_0)) - V_2_0*V_3_0*(g_23*np.cos(theta_23_0) + b_23*np.sin(theta_23_0))
d_q2_d_theta_3 = -V_2_0*V_3_0*(-g_23*np.cos(theta_23_0) - b_23*np.sin(theta_23_0))
d_q2_d_V1 = -V_2_0*(g_12*np.sin(theta_12_0) - b_12*np.cos(theta_12_0))
d_q2_d_V2 = V_1_0*(g_12*np.sin(theta_12_0) - b_12*np.cos(theta_12_0)) + V_3_0*(g_23*np.sin(theta_23_0) - b_23*np.cos(theta_23_0))
d_q2_d_V3 = -V_2_0*(g_23*np.sin(theta_23_0) - b_23*np.cos(theta_23_0))
H_0[5,:] = [d_q2_d_theta_2, d_q2_d_theta_3, d_q2_d_V1, d_q2_d_V2, d_q2_d_V3]

d_v1_d_v1 = 1
H_0[6,2] = d_v1_d_v1 
d_v2_d_v2 = 1
H_0[7,3] = d_v2_d_v2 

display(Latex(f'$H(x^0)=$'))
print(H_0)


## Gain matrix

In [ ]:
R = np.diag([0.008**2, 0.008**2, 0.010**2, 0.008**2, 0.008**2, 0.010**2, 0.004**2, 0.004**2])
G = H_0.T.dot(np.linalg.inv(R)).dot(H_0)
print(G)